In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re


In [11]:
max_pages = 500 + 1
# result_df = pd.DataFrame()

# website = 'www.apple.com'
website_list = [
    'www.apple.com', 'www.google.com', 'www.microsoft.com', 'www.facebook.com', 'www.twitter.com',
    'www.amazon.com', 'www.ebay.com', 'www.porkbun.com', 'www.godaddy.com', 'www.1password.com', 'nordvpn.com']


In [12]:
def get_reviews_from_page(page, website):
    return_df = pd.DataFrame()
    articles = page.find_all('article')

    for article in articles:
        country = article.find_next('aside').find_all('span')[-1].string

        article_section = article.find_next('section')
        stars = re.search(r'\d+', article_section.find('img')['alt']).group()
        date = article_section.find('time')['datetime']
        title = article_section.find('h2').string
        text_raw = article_section.find('p').contents
        text = ' '.join(line.text for line in text_raw if line.text != '')

        return_df = pd.concat(
            [return_df, pd.DataFrame(
                [{'site': website, 'date': date, 'stars': stars, 'title': title, 'text': text, 'location': country}])],
            ignore_index=True)

    return return_df


In [13]:
def scrape_reviews_for_website(url):

    result_df = pd.DataFrame()

    soup = BeautifulSoup(requests.get(
        f'https://www.trustpilot.com/review/' + url).text, 'lxml')
    max_available_pages = int(
        soup.find('a', {'name': 'pagination-button-last'}).find('span').string) + 1

    # already fetched site - no need for duplicate fetching
    result_df = pd.concat(
        [result_df, get_reviews_from_page(soup, url)], ignore_index=True)

    for index in range(2, max_available_pages if max_pages > max_available_pages else max_pages):
        page = BeautifulSoup(requests.get(
            f'https://www.trustpilot.com/review/' + url + '?page=' + str(index)).text, 'lxml')
        result_df = pd.concat(
            [result_df, get_reviews_from_page(page, url)], ignore_index=True)

    result_df.to_csv('data/' + url + ' - Trustpilot.csv')


In [14]:
for website in website_list:
    scrape_reviews_for_website(website)


KeyboardInterrupt: 